In [41]:
!pip install bert_score
from bert_score import score

In [ ]:
from sentence_transformers import SentenceTransformer, util, CrossEncoder

def compute_feature_cosine_similarity(text1, text2):
    # Load SBERT model
    model = SentenceTransformer('all-MiniLM-L6-v2')  # You can change this to other SBERT models

    # Encode the texts to get their embeddings
    emb1 = model.encode(text1, convert_to_tensor=True)
    emb2 = model.encode(text2, convert_to_tensor=True)

    # Compute cosine similarity
    similarity = util.pytorch_cos_sim(emb1, emb2)

    return similarity.item()

def compute_cross_encoder_similarity(text1: str, text2: str) -> float:
    # Load pre-trained Cross-Encoder model
    model = CrossEncoder('cross-encoder/stsb-roberta-large')  # You can use other CrossEncoder models too

    # Predict similarity score between 0 and 5
    score = model.predict([(text1, text2)])  # Must be a list of pairs

    return float(score[0])


In [ ]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration

def compute_bart_score(
    sources,
    candidates,
    model_name="facebook/bart-large-cnn"
):
    """
    Compute forward BARTScore for each (source, candidate) pair:
    BARTScore = - (cross_entropy_loss * sequence_length).

    - Higher is better: the candidate is more likely given the source.
    - You can also average by sequence length if you prefer a per-token measure.

    Args:
        sources (List[str]): List of source texts.
        candidates (List[str]): List of candidate (or "hypothesis") texts.
        model_name (str): Any valid BART seq2seq model on Hugging Face.
                          "facebook/bart-large-cnn" is often used for summarization tasks.

    Returns:
        List[float]: One BARTScore per (source, candidate) pair.
    """
    assert len(sources) == len(candidates), (
        "Sources and candidates must have the same length."
    )

    tokenizer = BartTokenizer.from_pretrained(model_name)
    model = BartForConditionalGeneration.from_pretrained(model_name)
    model.eval()  # put model in evaluation mode

    bart_scores = []

    for src_text, cand_text in zip(sources, candidates):
        with torch.no_grad():
            # Tokenize source as the "input"
            input_ids = tokenizer(src_text, return_tensors="pt", max_length=1024).input_ids

            # Tokenize candidate as the "label" to be generated
            labels = tokenizer(cand_text, return_tensors="pt", max_length=1024).input_ids

            # Forward pass: model outputs cross-entropy loss averaged across tokens
            outputs = model(input_ids=input_ids, labels=labels)
            loss = outputs.loss  # average cross-entropy per token

            # Convert to total negative log-likelihood
            # Multiply by the number of tokens to get the sum over the entire sequence
            n_tokens = labels.size(1)
            neg_log_likelihood = loss * n_tokens

            # BARTScore is typically the *negative* of this (since we want "higher is better")
            score = -neg_log_likelihood.item()
            bart_scores.append(score)

    return bart_scores

# ---- Example Usage ----
if __name__ == "__main__":
    sources = [text1]
    candidates = [text2]

    scores = compute_bart_score(sources, candidates)
    for i, s in enumerate(scores):
        print(f"Pair {i}: BARTScore = {s:.4f}")


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Pair 0: BARTScore = -4131.5752


In [ ]:
# text1 = "Large language models can generate human-like text."
# text2 = "Big language models produce text that appears human."

P, R, F1 = score(
    [text1],
    [text2],
    model_type="microsoft/deberta-xlarge-mnli"  # or another large model
)
print("Precision:", P.item())
print("Recall:", R.item())
print("F1:", F1.item())


Precision: 0.7462527751922607
Recall: 0.7545071840286255
F1: 0.7503572702407837


In [55]:
t4,t5,t6,t7,t8 = '''<proactive_followup>
Welcome back, Sarah! It's been two weeks since we set your weight loss goal. How have you been doing with replacing evening snacks with herbal tea and taking those 15-minute walks during your lunch break?
</proactive_followup>

Based on your history, I can see you've been implementing several strategies with varying effectiveness. Your pre-prepared healthy meals and prioritizing at least 10 minutes of exercise have been working well for you!

<follow_up>
Which of your plan elements has been the most challenging to maintain consistently over these past two weeks?
</follow_up>

<personalized_recommendation>
Based on your history, I suggest: Creating a specific evening routine that includes making your herbal tea at a set time before cravings typically hit.

This is because: Your implementation of the herbal tea strategy has shown "moderate" effectiveness, suggesting there's room for improvement in how you approach evening snacking.
</personalized_recommendation>

Would you like to share any wins you've experienced or adjustments you think might help with your current plan?''','''Hello! I'm here to help you achieve your health and wellbeing goals as your AI accountability buddy. I notice we're just getting started, so I'll need to gather some initial information from you.
<request_info> What specific health or wellbeing goal would you like to work on? Feel free to share as much detail as you're comfortable with about what you hope to achieve. </request_info>
Once I understand your goal, we can discuss your desired outcome, potential obstacles, and develop an actionable plan together. I'm here to provide support and personalized recommendations throughout your journey.''','''Welcome back, Sarah!
<proactive_followup> It's been two weeks since you set your weight loss goal. How have things been going with your plan to lose 20 pounds over the next 6 months? </proactive_followup>
Your Progress So Far
Looking at your history, I see you've been implementing several strategies:
Using herbal tea for late-night cravings (moderately effective)
Preparing healthy meals in advance (highly effective!)
Desk exercises when walks aren't possible (needs improvement)
Partial food tracking when busy (moderately effective)
Prioritizing quick exercise on busy days (highly effective!)
<personalized_recommendation> Based on history: Consider enhancing your desk exercise routine with specific timed intervals. Because: Your data shows this strategy has lower effectiveness compared to your meal prep and quick exercise priorities, which are working well. </personalized_recommendation>
Today's Check-in
I'd love to hear how the past few days have been for you:
Have you been able to maintain your 15-minute lunch walks?
How is the late-night snacking challenge going with the herbal tea strategy?
What's one win you've had this past week?
<suggested_plan> Based on your success patterns, consider these adjustments:
Continue your highly effective meal prep routine
Add a 2-minute desk stretch alarm every hour
Pair your herbal tea with a satisfying but low-calorie evening activity </suggested_plan>
Is there anything specific you'd like support with today?''','''<proactive_followup>
 Welcome back, Sarah! It's been two weeks since you set your goal to lose 20 pounds in 6 months. How have those plans been working for you, especially the herbal tea replacement for evening snacks and your lunchtime walks?
</proactive_followup>
I notice your most effective strategies so far have been using pre-prepared healthy meals when you're too tired to cook and prioritizing at least 10 minutes of exercise on busy days. The desk exercises when you can't take a lunch walk haven't been as effective.
<follow_up> Have you been able to consistently track your calorie intake using the mobile app? And how is your energy level feeling compared to when you started? </follow_up>''','''<proactive_followup>Welcome back, Sarah! It's been two weeks since you set your goal to lose 20 pounds in 6 months. I'm excited to see how you've been implementing your plan! I notice you've been testing different strategies, and it looks like pre-prepared healthy meals and prioritizing at least 10 minutes of exercise even on busy days have been working really well for you. How are you feeling about your progress so far?</proactive_followup>
<follow_up>How has the herbal tea strategy been working for your late-night snacking? Are you finding it easier to resist those cravings now?</follow_up>
<follow_up>I see your desk exercises weren't as effective. Would you like to explore other ways to incorporate movement during your workday that might fit better with your sedentary job?</follow_up>
<personalized_recommendation>Based on your history, I'd suggest continuing to prioritize meal prep on weekends since "using pre-prepared healthy meals when tired" has been highly effective for you. This strategy directly addresses both your lack of time obstacle and helps manage your calorie intake. Would you like some simple meal prep ideas that take less than 30 minutes to prepare?</personalized_recommendation>'''

In [52]:
P, R, F1 = score(
    [t4],
    [t7],
    model_type="microsoft/deberta-xlarge-mnli"  # or another large model
)
print("Precision:", P.item())
print("Recall:", R.item())
print("F1:", F1.item())

Precision: 0.6484244465827942
Recall: 0.6942765116691589
F1: 0.6705675721168518


In [57]:
P, R, F1 = score(
    [t7],
    [t8],
    model_type="microsoft/deberta-xlarge-mnli"  # or another large model
)
print("Precision:", P.item())
print("Recall:", R.item())
print("F1:", F1.item())

Precision: 0.7407271862030029
Recall: 0.679580807685852
F1: 0.7088378071784973


In [56]:
P, R, F1 = score(
    [t4],
    [t8],
    model_type="microsoft/deberta-xlarge-mnli"  # or another large model
)
print("Precision:", P.item())
print("Recall:", R.item())
print("F1:", F1.item())

Precision: 0.7070493698120117
Recall: 0.6942397356033325
F1: 0.7005860209465027


In [58]:
compute_feature_cosine_similarity(t4,t7), compute_feature_cosine_similarity(t4,t8 )

(0.7911161184310913, 0.8235524892807007)

In [42]:
t1, t2, t3 = '''<proactive_followup>Welcome back, Sarah! It's great to see you today. You're two weeks into your weight loss journey, and I can already see you've had some successes with your tea substitutions, meal prep, and starting to incorporate exercise. Those are fantastic building blocks!</proactive_followup>
I notice you've had particularly good results with pre-prepared meals and your 10-minute exercise commitment. These "high success" areas are great foundations to build upon.
Based on your history, it seems the desk exercises were more challenging. That's completely normal - finding what works best for your situation often takes some experimenting.
<personalized_recommendation>Since your meal prep and short exercise commitments are working well, let's build on that momentum. Consider combining your lunch walks with a buddy from work or a podcast you enjoy. This can make the habit more enjoyable and sustainable, especially with your sedentary job obstacle.</personalized_recommendation>
<personalized_recommendation>For evening snacking, since tea has been moderately successful, try enhancing this strategy by creating a small evening ritual around it - perhaps a special mug, a relaxing spot, and a few minutes of mindfulness as you enjoy it. This can help address the psychological component of late-night snacking.</personalized_recommendation>
<follow_up>How has your energy level been feeling since starting these changes, especially on days when you fit in that 10-minute exercise?</follow_up>''', '''<proactive_followup> Welcome back, Sarah! It's been a couple of weeks since you set your weight loss goal. How have your strategies been working, especially the herbal tea for late-night cravings and meal prepping? I see those have had varying effectiveness for you. </proactive_followup>
I notice you've been testing different strategies since May 2nd. Your pre-prepared healthy meals and prioritizing at least 10 minutes of exercise have been working well for you! The desk exercises when you can't walk during lunch seem to be less effective.
<follow_up> Which of your strategies has been the most helpful for you so far? And have you encountered any new obstacles I should know about? </follow_up>''','''<proactive_followup> Welcome back, Sarah! It's been two weeks since you set your goal to lose 20 pounds in 6 months. How are you feeling about your progress so far with replacing evening snacks, taking walks during lunch, meal prepping, and tracking your calories? </proactive_followup>
I see you've been implementing several strategies, with varying levels of success. Your pre-prepared healthy meals and prioritizing at least 10 minutes of exercise have been working well for you! The herbal tea for late-night cravings and meal tracking have been moderately effective, while desk exercises when you can't walk at lunch haven't been as helpful.
Would you like to discuss how any of these strategies are going, or would you prefer to update your plan based on what you've learned so far?'''

In [46]:
P, R, F1 = score(
    [t1],
    [t2],
    model_type="microsoft/deberta-xlarge-mnli"  # or another large model
)
print("Precision:", P.item())
print("Recall:", R.item())
print("F1:", F1.item())

Precision: 0.6158099174499512
Recall: 0.7051464915275574
F1: 0.6574572324752808


In [44]:
P, R, F1 = score(
    [t2],
    [t3],
    model_type="microsoft/deberta-xlarge-mnli"  # or another large model
)
print("Precision:", P.item())
print("Recall:", R.item())
print("F1:", F1.item())

Precision: 0.7837554216384888
Recall: 0.7692713737487793
F1: 0.7764458060264587


In [ ]:
compute_cross_encoder_similarity('hello my name is Eric', 'hello my name is Eric')

0.968502402305603

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer, util

# (Make sure you've run "nltk.download('punkt')" once before using sent_tokenize)

def naive_topk_sentence_summarize(text, k=20):
    """
    A simple, naive "top-k sentence" summarizer:
    Splits text into sentences, then picks the first k to represent the text.
    A more sophisticated approach might use a ranking or neural summarizer.
    """
    sentences = sent_tokenize(text)
    # Just pick the first k for demonstration
    summary = " ".join(sentences[:k])
    return summary

def compute_summary_based_similarity(text1, text2,
                                     model_name='all-MiniLM-L6-v2',
                                     summarizer=naive_topk_sentence_summarize):
    """
    Summarize each text, then compare the summaries using a bi-encoder model.
    """
    model = SentenceTransformer(model_name)

    # 1) Summarize both texts
    summary1 = summarizer(text1)
    summary2 = summarizer(text2)

    # 2) Encode summaries
    emb1 = model.encode(summary1, convert_to_tensor=True)
    emb2 = model.encode(summary2, convert_to_tensor=True)

    # 3) Compute cosine similarity
    similarity = util.cos_sim(emb1, emb2).item()
    return similarity


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

def compute_chunked_cosine_similarity(text1, text2,
                                      model_name='all-MiniLM-L6-v2',
                                      chunk_size=256,
                                      overlap=0):
    """
    Splits each text into chunks, encodes with SBERT, and computes average embedding.
    Returns the cosine similarity between the two averaged embeddings.

    text1, text2: Long text inputs (strings)
    model_name: Name of the SBERT model to use
    chunk_size: Max tokens/words per chunk (approx)
    overlap: Overlap in tokens/words between chunks to preserve context
    """
    # 1) Initialize model
    model = SentenceTransformer(model_name)

    # 2) Function to split text into chunks
    def chunk_text(text, size, overlap):
        words = text.split()
        chunks = []
        start = 0
        while start < len(words):
            end = min(start + size, len(words))
            chunk = " ".join(words[start:end])
            chunks.append(chunk)
            start += (size - overlap)  # Slide window
        return chunks

    # 3) Encode all chunks and average the embeddings
    def encode_and_average(text):
        segments = chunk_text(text, chunk_size, overlap)
        embeddings = model.encode(segments, convert_to_tensor=True)
        avg_embedding = embeddings.mean(dim=0)  # average across all chunks
        return avg_embedding

    # 4) Get average embeddings for each text
    emb1 = encode_and_average(text1)
    emb2 = encode_and_average(text2)

    # 5) Compute cosine similarity
    similarity = util.cos_sim(emb1, emb2).item()
    return similarity


In [ ]:
sim_chunked = compute_chunked_cosine_similarity(text1, text2)
print(f"Chunked SBERT similarity: {sim_chunked:.4f}")

Chunked SBERT similarity: 0.8973


In [ ]:
text1, text2 = '''Indemnification/Liability Clause Analysis
Looking at the indemnification clause between AlphaTech and Better Health, I see several important elements to analyze from a practical legal perspective:
Current Clause Components

AlphaTech indemnifies Better Health against losses arising from their services
Specific coverage includes data breach costs, IP infringement, and legal fees
Liability cap set at two times the total fees paid to AlphaTech

Weaknesses for Better Health
The liability cap is the most concerning element. In today's regulatory environment, data breach costs can far exceed two times the contract value, especially in healthcare where:

HIPAA violations can reach $1.5 million per violation category annually
State laws may impose additional penalties (e.g., CCPA allows $7,500 per intentional violation)
Class action settlements for healthcare data breaches often reach tens or hundreds of millions
Remediation costs including notification, credit monitoring, and forensics can be enormous

Recommended Revisions

Carve out regulatory fines/penalties from the liability cap:
"AlphaTech's total liability shall not exceed two times the total fees paid, except for regulatory fines and penalties resulting from AlphaTech's negligence or willful misconduct, which shall not be subject to this limitation."
Include specific language for PHI/PII breaches:
"In the event of unauthorized disclosure of Protected Health Information or Personally Identifiable Information caused by AlphaTech's failure to comply with security requirements, AlphaTech shall be responsible for all resulting regulatory penalties and remediation costs."
Add cyber insurance requirement:
"AlphaTech shall maintain cyber liability insurance with coverage of at least $10 million per occurrence."

Negotiation Strategies

Use market standards as leverage: Note that standard enterprise healthcare vendor contracts increasingly exclude regulatory penalties from liability caps.
Tiered approach: If AlphaTech resists unlimited liability for breaches, propose a separate, higher cap specifically for data breaches (10x contract value).
Provide concrete examples: Share anonymized actual breach costs from similar incidents to demonstrate the realistic financial exposure.
Focus on proportionality: Frame the discussion around risk allocation - who is better positioned to prevent and manage the risk should bear more of the potential cost.

This clause as currently written leaves Better Health with significant unhedged risk exposure that could threaten the organization's financial stability in a serious breach scenario.''', '''Analysis of Indemnification and Limitation of Liability Clause
Assessment of Current Protection
The clause as written provides basic indemnification coverage but has significant shortcomings for Better Health:
The liability cap of "two times the total fees paid" could be grossly inadequate for a healthcare data breach. In healthcare, data breaches can result in:

HIPAA penalties up to $1.5 million per violation category annually
State attorney general actions
Class action lawsuits from affected patients
Notification and credit monitoring costs for thousands or millions of patients
Reputational damage and business interruption

For context, major healthcare data breaches have resulted in settlements and costs well into the tens or hundreds of millions of dollars. If the contract value is relatively small (e.g., $500,000), the 2x cap would limit AlphaTech's liability to $1 million, which might cover only a fraction of actual damages from a significant breach.
Recommended Revisions

Exclude certain categories from the liability cap:
"AlphaTech's total liability shall not exceed an amount equal to two times the total fees paid by Better Health under this Agreement, PROVIDED HOWEVER that this limitation shall not apply to: (i) AlphaTech's indemnification obligations for data breaches involving Protected Health Information as defined by HIPAA; (ii) AlphaTech's indemnification obligations for regulatory fines and penalties; (iii) AlphaTech's indemnification obligations for third-party IP infringement claims; or (iv) damages arising from AlphaTech's gross negligence, willful misconduct, or fraud."
Add specific insurance requirements:
"AlphaTech shall maintain, at its own expense, cyber liability insurance coverage with limits not less than $10 million per occurrence and technology errors and omissions insurance with limits not less than $5 million per claim, naming Better Health as an additional insured."
Add regulatory compliance specificity:
"AlphaTech acknowledges that it will be functioning as a Business Associate under HIPAA and agrees to indemnify Better Health for any penalties, fines, or costs arising from AlphaTech's failure to comply with applicable privacy and security regulations."

Negotiation Strategies

Benchmark approach: Obtain market data on standard liability caps in healthcare technology contracts (typically 3-5x contract value or uncapped for certain categories) and present this during negotiations.
Tiered approach: Propose different liability caps for different categories of risk:

General performance issues: 2x fees
Security incidents not involving PHI: 5x fees
PHI data breaches: 10x fees or uncapped
IP infringement: Uncapped


Risk assessment leverage: Commission a third-party risk assessment of the project scope that quantifies potential breach costs. Use these figures to justify why the current cap is inadequate.
Escrow provision: Request source code and documentation escrow as additional protection, allowing Better Health to maintain systems if AlphaTech is unable to fulfill obligations.
Performance-based adjustment: Propose that the liability cap increases over time if certain security metrics or performance benchmarks aren't met.

In healthcare technology contracts, it's particularly important that liability protections align with the actual risk profile. Given the sensitive nature of healthcare data and the severe potential consequences of a breach, Better Health would be justified in seeking stronger protections than the current clause provides.'''

compute_feature_cosine_similarity(text1, text2), compute_cross_encoder_similarity(text1, text2)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

(0.906212329864502, 0.6961562633514404)

In [ ]:
text1, text2, text3 = '''Protection Adequacy for Severe Data Breaches
Relevant Clause: "AlphaTech shall indemnify... Better Health... against any losses... arising out of... services... including... data breach costs..."
Interpretation: The clause creates a broad obligation for AlphaTech to compensate Better Health for losses resulting from data breaches connected to their services. The inclusion of "data breach costs" demonstrates acknowledgment of this specific risk category.
Implications: While the inclusion of data breach indemnification is positive, the language lacks specificity about what constitutes "data breach costs." This ambiguity could lead to disputes over whether certain expenses (like forensic investigation, notification costs, credit monitoring, or public relations management) are covered.
Improvement Needed: The protection is fundamentally undermined by the liability cap, as healthcare data breaches frequently result in damages far exceeding twice the contract value. A 2023 IBM report indicated the average healthcare data breach cost reached $10.93 million - likely far above the contract value's double amount.
Suggested Revisions for Regulatory Penalties
Relevant Clause: "AlphaTech's total liability shall not exceed... two times the total fees paid"
Interpretation: This creates a hard cap on AlphaTech's financial responsibility, regardless of the actual damages incurred. Notably, there are no exceptions to this cap for regulatory penalties, which represents a significant gap in protection.
Implications: Regulatory penalties for healthcare data breaches can be substantial:

HIPAA violations can reach $1.5 million per violation category annually
State data protection laws (like CCPA/CPRA in California) impose additional penalties
International regulations (like GDPR) may apply depending on data subjects

Recommended Revisions:

Add specific carve-out: "Notwithstanding the foregoing, the liability cap shall not apply to regulatory fines and penalties resulting from AlphaTech's non-compliance with applicable laws and regulations."
Include separate breach indemnification: "In the event of a data breach involving Protected Health Information, AlphaTech shall be responsible for all resulting regulatory penalties, notification costs, credit monitoring services, and remediation expenses."
Add specification: "AlphaTech's indemnification obligations include costs related to OCR investigations, state attorney general inquiries, and compliance with breach notification laws."

Negotiation Strategies for Better Health
Relevant Clause: "AlphaTech shall indemnify... Better Health... against any losses... AlphaTech's total liability shall not exceed... two times the total fees paid"
Interpretation: The clause creates tension between a broad indemnification promise and a strict liability limitation.
Implications: Better Health has significant leverage in this negotiation because:

As a healthcare entity, it faces unique regulatory obligations under HIPAA/HITECH
The risks of a data breach extend beyond direct financial costs to reputation and patient trust
Healthcare vendors increasingly accept stronger indemnification provisions as market standard

Effective Negotiation Strategies:

Evidence-based approach: Present documentation of recent healthcare data breach settlements and costs to demonstrate the inadequacy of the cap.
Tiered liability structure: Propose different caps for different categories of breach (e.g., higher cap for PHI breaches versus general service issues).
Insurance verification: Request proof of AlphaTech's cyber insurance coverage and ensure coverage aligns with potential exposure.
Industry standards leverage: Provide examples of standard indemnification terms from other healthcare technology vendors.
Compromise position: If AlphaTech resists unlimited liability, propose a significantly higher cap specifically for data breaches (e.g., 5-10x the standard cap).

Real-World Implications of the Liability Cap
Relevant Clause: "AlphaTech's total liability shall not exceed... two times the total fees paid"
Interpretation: This creates a predictable maximum financial exposure for AlphaTech but transfers the risk of catastrophic breaches to Better Health.
Implications:

Financial exposure gap: In a major breach, Better Health could face millions in uncovered costs while AlphaTech's responsibility remains limited.
Misaligned incentives: With limited financial exposure, AlphaTech may not prioritize security investments proportionate to the actual risk.
Insurance coordination challenges: Better Health's cyber insurance may have subrogation rights that conflict with the liability cap.
Business continuity concerns: A severe, uncovered breach could threaten Better Health's financial stability.

Real-world case relevance:

The 2023 McLaren Health breach affected 2.2 million patients with estimated costs exceeding $15 million
Change Healthcare's 2024 ransomware incident resulted in widespread healthcare disruption with costs continuing to mount
The Anthem settlement of $115 million demonstrates the scale of potential class action liability

Recommendation Summary

Remove or create exceptions to the liability cap for:

Regulatory penalties and fines
Data breach notification and remediation costs
Claims involving PHI/PII


Enhance specificity in the indemnification language to explicitly cover:

Forensic investigation expenses
Credit monitoring services
Call center support for affected individuals
Public relations management during breach response


Add insurance requirements including:

Minimum cyber liability coverage of $10 million
Naming Better Health as an additional insured
Non-cancellation without notice provisions


Include breach response protocols that clearly define:

Notification timeframes
Evidence preservation requirements
Coordination of public communications


Add representation that AlphaTech maintains a comprehensive security program aligned with HIPAA Security Rule requirements and industry best practices.

These modifications would significantly strengthen Better Health's protection and better align the risk allocation with each party's ability to control and mitigate potential data breach issues.''', '''Point 1: Protection adequacy for severe data breaches
Relevant parts of the clause:

"AlphaTech shall indemnify and hold Better Health harmless from and against any losses, damages, liabilities, claims, or expenses arising out of or relating to the provision of services hereunder, including but not limited to data breach costs..."
"AlphaTech's total liability shall not exceed an amount equal to two times the total fees paid by Better Health under this Agreement."

Interpretation:
The first part establishes a broad indemnification obligation for AlphaTech, covering various categories of potential damages, including data breach costs. This is favorable for Better Health as it shifts responsibility for data breach costs to AlphaTech. However, the second part establishes a cap on AlphaTech's total liability at two times the contract value.
Potential implications:
In the healthcare context, data breaches can be extraordinarily costly, potentially including:

HIPAA penalties (up to $1.5 million per violation category per year)
State data breach notification costs (which can run into millions for large breaches)
Credit monitoring services for affected individuals
Litigation costs and settlements from class action lawsuits
Regulatory investigations and fines
Reputational damage and remediation costs
Business disruption costs

The cap at two times contract value could be grossly inadequate if:

The contract value is relatively small compared to the potential breach impact
The breach affects a large number of patient records
Multiple regulatory frameworks apply (HIPAA, GDPR, state laws)
The breach results from gross negligence or willful misconduct

Initial thoughts on improvements:

Exclude data breaches from the liability cap entirely, or set a much higher specific cap for breaches
Add specific language addressing regulatory penalties and fines
Include carve-outs for gross negligence, willful misconduct, and fraud
Consider adding insurance requirements to provide additional protection

Point 2: Suggested revisions for regulatory penalties
Relevant parts of the clause:

"...any losses, damages, liabilities, claims, or expenses arising out of or relating to the provision of services hereunder, including but not limited to data breach costs..."
"AlphaTech's total liability shall not exceed an amount equal to two times the total fees paid..."

Interpretation:
While the indemnification provision broadly covers "liabilities," which could include regulatory penalties, the liability cap would still apply to these penalties. This creates significant risk for Better Health, as they could be exposed to regulatory penalties beyond the cap amount.
Potential implications:
Healthcare regulatory fines and penalties can be substantial:

HHS OCR has imposed HIPAA penalties in the millions of dollars
State attorneys general can pursue separate actions
Multiple regulatory frameworks might apply simultaneously
Penalties can be assessed per violation, per record, which can quickly escalate to enormous sums
Regulatory frameworks often hold the covered entity (Better Health) primarily responsible, regardless of contractual arrangements with vendors

The current clause does not adequately address regulatory penalties specifically, and the cap may leave Better Health with significant uncovered liability.
Initial thoughts on improvements:

Explicitly exclude regulatory fines and penalties from the liability cap
Add specific language acknowledging AlphaTech's role as a Business Associate under HIPAA
Include provisions requiring AlphaTech to cooperate with regulatory investigations
Add language specifically addressing indemnification for penalties resulting from AlphaTech's failure to comply with applicable regulations
Consider a separate, higher cap specifically for regulatory penalties
Add insurance requirements specifically covering regulatory defense and penalties

Point 3: Negotiation strategies for Better Health
Relevant parts of the clause:

The entire indemnification and limitation of liability provision

Interpretation:
The current clause presents a balanced approach on its face (broad indemnification with a reasonable-sounding cap), but in the healthcare context, it likely shifts too much risk to Better Health given the potential costs of data breaches.
Potential implications:
Better Health may face significant uncovered liabilities if they accept the clause as written. AlphaTech may be unwilling to accept unlimited liability or a very high cap, as this could represent an existential business risk.
Initial thoughts on strategies:

Risk-based approach: Conduct a risk assessment to quantify potential breach costs and use that to justify a higher cap or exclusions from the cap.
Industry standards approach: Present market data on typical liability provisions in healthcare technology contracts to demonstrate that two times fees is below market standard.
Carve-out approach: Accept the general liability cap but negotiate specific carve-outs for high-risk scenarios.
Insurance-focused approach: Shift the conversation from caps to insurance requirements, ensuring AlphaTech maintains adequate coverage.
Tiered approach: Negotiate different liability caps for different categories of breaches or claims.
Performance-based approach: Link liability caps to compliance with specific security standards and requirements.
Contract value approach: If the contract value is small relative to potential risk, negotiate a minimum dollar amount for the cap rather than a multiple of fees.

Point 4: Real-world implications of the liability cap and indemnification
Relevant parts of the clause:

The entire indemnification and limitation of liability provision

Interpretation:
In practice, this clause creates a significant gap between potential damages and recoverable amounts, particularly in serious breach scenarios.
Potential implications:
Real-world healthcare data breaches have resulted in costs far exceeding typical contract values:

The Anthem breach settlement was $115 million
The Equifax breach (not healthcare but illustrative) resulted in a $700 million settlement
Providence Health's 2022 breach impacted 2.1 million patients
A major health system might pay a few million dollars for a software contract, but face tens or hundreds of millions in breach costs

The gap between the liability cap and potential damages means Better Health could be left with substantial uncovered costs in a breach scenario, even with indemnification.
Initial thoughts on improvements:

Include case studies or examples in negotiation materials to illustrate the real-world risk
Consider how cyber insurance policies interact with the contract indemnification
Address specific components of breach response that might not be explicitly covered in the current language (forensic investigation, public relations costs, executive time, etc.)
Add provisions addressing data recovery and business continuity in breach scenarios
Include specific performance requirements for breach notification timelines and cooperation
</clause_analysis>

Comprehensive Analysis of Indemnification and Liability Clause
1. Protection Adequacy
The current clause provides inadequate protection for Better Health in cases of severe data breaches for several critical reasons:
First, while the indemnification language is appropriately broad ("any losses, damages, liabilities, claims, or expenses"), the liability cap severely undermines this protection. A cap of "two times the total fees paid" is significantly insufficient in the healthcare context, where data breaches routinely result in damages that far exceed the value of the underlying contract.
For perspective, recent healthcare data breaches have resulted in costs that would likely overwhelm this cap:

The average cost per breached healthcare record is approximately $429 (IBM Security Cost of a Data Breach Report)
Settlement amounts commonly reach tens of millions of dollars
Regulatory fines can apply per violation, quickly accumulating to substantial sums

The clause fails to distinguish between routine contractual damages and extraordinary damages resulting from data breaches or IP infringement. This creates a fundamental disconnect between the protection promised in the first sentence and the limitation imposed in the second.
2. Suggested Revisions
To address the inadequacies, particularly regarding regulatory penalties, I recommend the following specific revisions:

Carve out exceptions to the liability cap:
AlphaTech's total liability shall not exceed an amount equal to two times the total fees paid by Better Health under this Agreement, EXCEPT THAT such limitation shall not apply to: (i) AlphaTech's indemnification obligations for breaches involving Protected Health Information; (ii) regulatory fines and penalties; (iii) third-party IP infringement claims; or (iv) damages arising from AlphaTech's gross negligence, willful misconduct, or fraud.

Add specific regulatory compliance language:
AlphaTech acknowledges that it will be functioning as a Business Associate under HIPAA and agrees to indemnify Better Health for any penalties, fines, or costs arising from AlphaTech's failure to comply with applicable privacy and security regulations, including but not limited to HIPAA, HITECH, and applicable state privacy laws.

Include insurance requirements:
AlphaTech shall maintain, at its own expense: (i) cyber liability insurance with limits not less than $10 million per occurrence; (ii) technology errors and omissions insurance with limits not less than $5 million per claim; and (iii) commercial general liability insurance with limits not less than $5 million per occurrence. Such policies shall name Better Health as an additional insured, and certificates evidencing such coverage shall be provided to Better Health upon request.

Add breach response specifications:
In the event of a data breach involving Better Health's data, AlphaTech shall, at its own expense, cooperate fully with Better Health's breach response efforts, including forensic investigation, notification of affected individuals, regulatory reporting, and provision of credit monitoring services as required by applicable law or as reasonably requested by Better Health.


3. Negotiation Strategies
For Better Health to secure more favorable terms, I recommend these negotiation strategies:

Risk quantification approach: Conduct an assessment that calculates potential breach costs based on the number of patient records involved, applying the average per-record cost ($429). Present this concrete figure to demonstrate why the current cap is disproportionate to the actual risk.
Industry benchmark strategy: Collect examples of similar healthcare technology agreements with higher liability caps (typically 3-5x contract value) or specific carve-outs for data breaches and regulatory fines. Present these as market standards that AlphaTech should meet.
Tiered liability structure: Propose a compromise with different liability thresholds:

General service issues: 2x fees
Security incidents not involving PHI: 3x fees
PHI data breaches: 5x fees or $10 million, whichever is greater
IP infringement: Uncapped


Insurance-focused compromise: If AlphaTech resists higher caps, shift focus to comprehensive insurance requirements with Better Health as an additional insured, which can provide protection beyond the contractual cap.
Regulatory compliance emphasis: Frame the discussion around AlphaTech's obligations as a Business Associate under HIPAA, emphasizing that these are non-negotiable regulatory requirements rather than merely contractual terms.

4. Real-world Implications
In practice, the current liability cap creates significant uncovered risk exposure for Better Health:

Regulatory gap: HIPAA penalties can reach $1.5 million per violation category annually. If the contract value is $2 million, the current cap ($4 million) could be exhausted by regulatory penalties alone, leaving nothing for other damages.
Scale disparity: A breach affecting 100,000 patient records could generate approximately $42.9 million in costs based on industry averages. The gap between this figure and the liability cap represents uncovered risk that Better Health would bear.
Indemnification conflict: The broad indemnification promise in the first sentence creates an expectation of comprehensive protection, but the cap effectively nullifies this for any significant breach, creating contractual ambiguity that could lead to disputes.
Practical recovery limitations: Even with a successful claim against AlphaTech, Better Health may face collection challenges if AlphaTech lacks adequate insurance or financial resources to satisfy the claim up to the cap.
Breach response constraints: Without specific cooperation requirements, Better Health might face difficulties getting timely assistance from AlphaTech during breach response, potentially exacerbating damages.

Conclusion
The current clause creates a dangerous illusion of protection through broad indemnification language while simultaneously imposing a liability cap that would likely prove inadequate in any significant data breach scenario. Better Health should prioritize excluding data breaches and regulatory penalties from the liability cap entirely, or at minimum, securing a substantially higher cap specific to these high-risk categories. Additionally, explicit insurance requirements provide a crucial secondary layer of protection. Given the extraordinary sensitivity of healthcare data and the severe consequences of breaches, these revisions are not merely preferable but essential for responsible risk management.''','''AlphaTech-Better Health Indemnification Clause Analysis
As you've requested, I'll analyze the indemnification and liability cap clauses from the contract between AlphaTech and Better Health. I'll structure my analysis according to your requirements.
Protection Adequacy for Severe Data Breaches
Relevant clause components:

"AlphaTech indemnifies Better Health against losses from data breaches..."
"AlphaTech's liability is capped at twice the total fees paid"

Interpretation:
The indemnification provision offers Better Health protection against financial losses resulting from data breaches, which is essential in today's digital environment. However, the liability cap creates a significant limitation on this protection.
Implications:
In cases of severe data breaches, damages can far exceed twice the contract value. This includes:

Regulatory fines (which can reach millions of dollars)
Class action lawsuits from affected individuals
Remediation costs
Reputational damage costs
Business interruption losses

Assessment:
The current protection is inadequate for severe data breaches. While indemnification for data breaches is included, the liability cap significantly undermines this protection in worst-case scenarios, especially if Better Health is handling sensitive personal or health information subject to regulations like HIPAA.
Suggested Revisions for Regulatory Penalties
Relevant clause components:

"AlphaTech indemnifies Better Health against losses from data breaches..."
"AlphaTech's liability is capped at twice the total fees paid"

Interpretation:
The current language doesn't specifically address regulatory penalties, which represent a major financial risk in data breach scenarios. The cap applies universally without carve-outs for specific types of damages.
Suggested revisions:

Create a carve-out for regulatory penalties: "Notwithstanding the foregoing cap on liability, AlphaTech shall indemnify Better Health for the full amount of any regulatory fines or penalties imposed as a direct result of AlphaTech's breach of this Agreement or applicable law."
Add specific language addressing notification costs: "AlphaTech shall bear all costs associated with legally required breach notifications to affected individuals and regulatory authorities resulting from data breaches caused by AlphaTech."
Include cyber insurance requirements: "AlphaTech shall maintain cyber liability insurance coverage with limits of no less than $10 million per occurrence, naming Better Health as an additional insured."

Negotiation Strategies for Better Health
Relevant clause components:

"AlphaTech indemnifies Better Health against losses from data breaches, IP infringement claims, and related costs"
"AlphaTech's liability is capped at twice the total fees paid"

Interpretation:
The current language gives Better Health some leverage in negotiations since data breach and IP infringement indemnification are explicitly included. However, the blanket liability cap significantly reduces this protection.
Negotiation strategies:

Tiered liability caps: "Better Health should propose different caps for different types of breaches. For example:

Standard cap of twice fees paid for general damages
Higher cap (5x fees) for data breaches involving personally identifiable information
Uncapped liability for willful or grossly negligent acts


Carve-outs from the cap: Negotiate complete exclusions from the liability cap for:

Regulatory fines and penalties
Third-party claims for data breaches
IP infringement claims
Breaches of confidentiality obligations


Insurance requirements: Request proof of AlphaTech's cyber insurance coverage with adequate limits, and negotiate to be named as an additional insured.
Benchmarking approach: Research and present industry standards for similar contracts to demonstrate that the proposed cap is below market expectations.

Real-World Implications of the Liability Cap
Relevant clause components:

"AlphaTech's liability is capped at twice the total fees paid"

Interpretation:
This cap creates a significant disconnect between potential damages and available remedies, particularly if the contract value is relatively small compared to the value of data being processed.
Real-world implications:

Financial exposure: If the contract value is $500,000, liability is capped at $1 million. However, a significant data breach could cause damages in the tens of millions.
Regulatory consequences: Regulators like the FTC, OCR (for HIPAA), or EU authorities (for GDPR) can impose penalties in the millions regardless of what the contract stipulates between the parties.
Insurance considerations: Better Health's cyber insurance provider may have subrogation rights that are effectively limited by this cap, potentially affecting coverage terms or costs.
Risk transfer failure: The purpose of indemnification is to transfer risk to the party best positioned to prevent harm (AlphaTech). This cap undermines this risk transfer mechanism.
Incentive misalignment: A low liability cap may reduce AlphaTech's financial incentive to invest in robust security measures.

Recommendation Summary
Better Health should:

Seek specific carve-outs from the liability cap for regulatory penalties, third-party claims, IP infringement, and willful/grossly negligent acts.
Negotiate tiered liability caps that reflect the different levels of risk associated with different types of breaches.
Require appropriate cyber liability insurance with Better Health named as an additional insured.
Add specific language addressing data breach response costs to ensure AlphaTech bears responsibility for legally required notifications and remediation efforts.
Consider the total value at risk when evaluating the contract, not just the contract price – if the data being handled is highly sensitive or voluminous, a cap of twice the fees paid is likely inadequate.

This analysis is provided for informational purposes only. As this involves complex legal matters with significant potential implications, I strongly recommend consulting with qualified legal counsel before finalizing any contract terms.'''

In [ ]:
compute_feature_cosine_similarity(text1, text2), compute_chunked_cosine_similarity(text1, text2), compute_cross_encoder_similarity(text1, text2)

(0.9309296607971191, 0.9376736879348755, 0.6339088082313538)

In [ ]:
compute_cross_encoder_similarity(text1, text2)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

0.676011323928833

In [ ]:
sim_summary = compute_summary_based_similarity(text1, text2)
print(f"Summary-based similarity: {sim_summary:.4f}")

Summary-based similarity: 0.9062


In [ ]:
from sentence_transformers import SentenceTransformer, util, CrossEncoder
import anthropic

client = anthropic.Anthropic(api_key="")  # or use os.environ["ANTHROPIC_API_KEY"]

def compute_feature_cosine_similarity(text1, text2):
    # Load SBERT model
    model = SentenceTransformer('all-MiniLM-L6-v2')  # You can change this to other SBERT models

    # Encode the texts to get their embeddings
    emb1 = model.encode(text1, convert_to_tensor=True)
    emb2 = model.encode(text2, convert_to_tensor=True)

    # Compute cosine similarity
    similarity = util.pytorch_cos_sim(emb1, emb2)

    return similarity.item()


def compute_cross_encoder_similarity(text1: str, text2: str) -> float:
    # Load pre-trained Cross-Encoder model
    model = CrossEncoder('cross-encoder/stsb-roberta-base')  # You can use other CrossEncoder models too

    # Predict similarity score between 0 and 5
    score = model.predict([(text1, text2)])  # Must be a list of pairs

    return float(score[0])

def get_claude_similarity_score(text1: str, text2: str) -> str:
    prompt = f"""
                Rate the semantic similarity between the following two texts on a scale from 1 to 10:
                (1 = Not similar at all, 10 = Very similar in meaning)

                Text 1: "{text1}"
                Text 2: "{text2}"

                Respond with only a single number between 1 and 10.
              """

    response = client.messages.create(
        model="claude-3-haiku-20240307",  # or "opus", "sonnet" based on your tier
        max_tokens=10,
        temperature=0,
        system="You are an expert in semantic similarity and NLP.",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.content[0].text.strip()


if __name__ == "__main__":
    # Example texts
    text1 = "A group of friends are planning a trip to the mountains."
    text2 = "Several people are organizing a vacation in the hills."

    score1 = compute_feature_cosine_similarity(text1, text2)
    score2 = compute_cross_encoder_similarity(text1, text2)
    score3 = get_claude_similarity_score(text1, text2)

    print(f"Cosine similarity score: {score1:.4f}")
    print(f"Cross-Encoder score: {score2:.4f}")
    print(f"Claude similarity score (0-10): {score3}")

ModuleNotFoundError: No module named 'anthropic'

In [ ]:
!pip install anthropic